In [ ]:
## features
# - Description, MISC and Category
# - Sentiment analysis of user reviews
import pandas as pd
import json

# assuming your JSON is loaded into a variable called `data`
data = json.load(open("datasets\data_for_gpt-oss.json"))

df = pd.json_normalize(data, sep='_')

# Convert review_time (ms) to readable datetime
df['review_time'] = pd.to_datetime(df['review_time'], unit='ms')

print(df.head())
print(df.columns)

df = df[['meta_name', 'meta_address', 'meta_gmap_id', 'meta_description', 'meta_category', 'meta_avg_rating',
       'meta_num_of_reviews','review_user_id',
       'review_name', 'review_rating', 'review_text',
       'review_pics', 'review_resp', 'review_gmap_id',
       'meta_MISC_Service options', 'meta_MISC_Accessibility',
       'meta_MISC_Amenities', 'meta_MISC_Planning', 'review_resp_time',
       'review_resp_text', 'meta_MISC_Offerings', 'meta_MISC_Payments',
       'meta_MISC_From the business', 'meta_MISC_Health & safety',
       'meta_MISC_Highlights', 'meta_MISC_Popular for',
       'meta_MISC_Dining options', 'meta_MISC_Atmosphere', 'meta_MISC_Crowd',
       'meta_MISC_Lodging options', 'meta_MISC_Health and safety',
       'meta_MISC_Recycling']]

print(df.head())
print(df.columns)
print(df.info())

# # Select all MISC columns
# misc_cols = [col for col in df.columns if col.startswith("meta_MISC_")]

# # Replace NaN with "No" and flatten lists to comma-separated strings
# for col in misc_cols:
#     df[col] = df[col].fillna("No").apply(
#         lambda x: ", ".join(x) if isinstance(x, list) else x
#     )


misc_cols = [col for col in df.columns if col.startswith("meta_MISC_")]
# for col in misc_cols:
#     df[col] = df[col].fillna("").apply(
#         lambda x: ", ".join([c.strip() for c in x]) if isinstance(x, list) else str(x).strip()
#     )

# # When combining, also strip the final result
# df['business_features'] = (
#     df['meta_description'].fillna("").str.strip() + ", " +
#     df['meta_category'] + ", " +
#     df[misc_cols].apply(lambda row: " ".join(row.values.astype(str)), axis=1)
# ).str.strip()

# print(df['business_features'].unique())

def build_features(row):
    parts = []
    if row["meta_description"]:
        parts.append(row["meta_description"].strip())
    if row["meta_category"]:
        parts.append(row["meta_category"].strip())
    # MISC fields
    misc_texts = []
    for v in row[misc_cols]:
        if isinstance(v, list):
            # flatten list into comma-separated string
            misc_texts.append(", ".join([str(x).strip() for x in v if str(x).strip() not in ("", "nan")]))
        elif pd.notna(v) and str(v).strip() not in ("", "nan"):
            misc_texts.append(str(v).strip())
    
    if misc_texts:
        parts.append(" ".join(misc_texts))
    
    # Join all non-empty parts with ", "
    return ", ".join(parts)

df["business_features"] = df.apply(build_features, axis=1)




<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
C:\Users\USER\AppData\Local\Temp\ipykernel_21364\990005932.py:8: SyntaxWarning: invalid escape sequence '\d'
  data = json.load(open("datasets\data_for_gpt-oss.json"))


                     meta_name  \
0  Bear Creek Cabins & RV Park   
1  Bear Creek Cabins & RV Park   
2  Bear Creek Cabins & RV Park   
3  Bear Creek Cabins & RV Park   
4  Bear Creek Cabins & RV Park   

                                        meta_address  \
0  Bear Creek Cabins & RV Park, 3181 Richardson H...   
1  Bear Creek Cabins & RV Park, 3181 Richardson H...   
2  Bear Creek Cabins & RV Park, 3181 Richardson H...   
3  Bear Creek Cabins & RV Park, 3181 Richardson H...   
4  Bear Creek Cabins & RV Park, 3181 Richardson H...   

                            meta_gmap_id meta_description  meta_latitude  \
0  0x56b646ed2220b77f:0xd8975e316de80952             None      61.100644   
1  0x56b646ed2220b77f:0xd8975e316de80952             None      61.100644   
2  0x56b646ed2220b77f:0xd8975e316de80952             None      61.100644   
3  0x56b646ed2220b77f:0xd8975e316de80952             None      61.100644   
4  0x56b646ed2220b77f:0xd8975e316de80952             None      61.100644   

 

C:\Users\USER\AppData\Local\Temp\ipykernel_21364\990005932.py:8: SyntaxWarning: invalid escape sequence '\d'
  data = json.load(open("datasets\data_for_gpt-oss.json"))


AttributeError: 'list' object has no attribute 'strip'

In [ ]:
display(df['business_features'][1])

', RV park, Cabin rental agency, Campground,'

In [15]:
display(df['meta_description'])

0          
1          
2          
3          
4          
         ..
159995     
159996     
159997     
159998     
159999     
Name: meta_description, Length: 160000, dtype: object